# Optimisation hyperparamètres des modèles prometteurs

On va optimiser les hyperparamètres des modèles prometteurs que nous avons identifiés dans le notebook précédent. 

On va utiliser des méthodes de recherche d'hyperparamètres par validation croisée pour éviter le surapprentissage.

On va utiliser les méthodes bayésiennes pour optimiser les hyperparamètres des modèles de manière intelligente.

In [3]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv('train.csv')

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# On sélectionne les colonnes d'entrée (features) et la cible
X = data.drop(columns=['Cover_Type'])
y = data['Cover_Type']

# On sélectionne les colonnes d'entrée (features) et la cible
X = data.drop(columns=['Cover_Type'])
y = data['Cover_Type']

# On divise les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# On normalise les données continues 
from sklearn.preprocessing import StandardScaler

continuous_columns = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 
                      'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 
                      'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']
scaler = StandardScaler()

# On applique le scaler uniquement sur les colonnes continues
X_train[continuous_columns] = scaler.fit_transform(X_train[continuous_columns])
X_test[continuous_columns] = scaler.transform(X_test[continuous_columns])

In [2]:
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

## Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier

# On définit l'espace de recherche pour Random Forest
space_rf = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'max_depth': hp.choice('max_depth', [5, 10, 15, 20, None]),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4]),
    'max_features': hp.choice('max_features', ['sqrt', 'log2'])  # Correction : 'auto' remplacé par 'sqrt'
}

# On crée la fonction objectif pour Random Forest
def objective_rf(params):
    # On initialise le modèle avec les paramètres proposés
    model = RandomForestClassifier(**params, random_state=42)
    # On effectue une validation croisée pour évaluer la performance
    score = cross_val_score(model, X_train, y_train, cv=7, scoring='accuracy').mean()
    # On retourne la perte (score négatif) car Hyperopt minimise cette fonction
    return {'loss': -score, 'status': STATUS_OK}

# On lance Hyperopt pour optimiser les hyperparamètres de Random Forest
trials_rf = Trials()
best_params_rf = fmin(fn=objective_rf, space=space_rf, algo=tpe.suggest, max_evals=50, trials=trials_rf)
print("Meilleurs paramètres Random Forest :", best_params_rf)

100%|██████████| 50/50 [27:22<00:00, 32.86s/trial, best loss: -0.8746220710506425]  
Meilleurs paramètres Random Forest : {'max_depth': 4, 'max_features': 0, 'min_samples_leaf': 0, 'min_samples_split': 0, 'n_estimators': 2}


On teste ensuite le modèle avec les hyperparamètres optimisés sur les données de test.

In [19]:
# Traduction des indices en valeurs concrètes
best_rf_params = {
    'max_depth': [5, 10, 15, 20, None][4],           
    'max_features': ['sqrt', 'log2'][0],             
    'min_samples_leaf': [1, 2, 4][0],                
    'min_samples_split': [2, 5, 10][0],              
    'n_estimators': [100, 200, 300, 400, 500][2]     
}

# Initialiser le modèle avec les valeurs concrètes
rf_model = RandomForestClassifier(**best_rf_params, random_state=42)

# Entraîner le modèle sur l'ensemble d'entraînement
rf_model.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred_rf = rf_model.predict(X_test)

# Évaluer les performances du modèle
from sklearn.metrics import accuracy_score, classification_report
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Précision Random Forest :", accuracy_rf)
print("Rapport de classification Random Forest :\n", classification_report(y_test, y_pred_rf))

Précision Random Forest : 0.8763227513227513
Rapport de classification Random Forest :
               precision    recall  f1-score   support

           1       0.79      0.78      0.79       648
           2       0.80      0.71      0.75       648
           3       0.85      0.88      0.87       648
           4       0.95      0.98      0.96       648
           5       0.92      0.95      0.93       648
           6       0.88      0.88      0.88       648
           7       0.93      0.95      0.94       648

    accuracy                           0.88      4536
   macro avg       0.87      0.88      0.87      4536
weighted avg       0.87      0.88      0.87      4536



## XGBoost

In [21]:
import xgboost as xgb

y_train_adj = y_train - 1
y_test_adj = y_test - 1

# On définit l'espace de recherche pour XGBoost
space_xgb = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'max_depth': hp.choice('max_depth', [3, 6, 9, 12]),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'gamma': hp.uniform('gamma', 0, 0.5)
}

# On crée la fonction objectif pour XGBoost
def objective_xgb(params):
    # On initialise XGBoost avec les paramètres proposés
    model = xgb.XGBClassifier(objective='multi:softmax', num_class=7, **params, random_state=42)
    # On utilise une validation croisée pour calculer la précision
    score = cross_val_score(model, X_train, y_train_adj, cv=7, scoring='accuracy').mean()
    # On retourne la perte (score négatif) car Hyperopt minimise cette fonction
    return {'loss': -score, 'status': STATUS_OK}

# On lance l'optimisation avec Hyperopt pour XGBoost
trials_xgb = Trials()
best_params_xgb = fmin(fn=objective_xgb, space=space_xgb, algo=tpe.suggest, max_evals=50, trials=trials_xgb)
print("Meilleurs paramètres XGBoost :", best_params_xgb)

100%|██████████| 50/50 [24:26<00:00, 29.34s/trial, best loss: -0.8828420256991685]   
Meilleurs paramètres XGBoost : {'colsample_bytree': 0.9284956459967804, 'gamma': 0.04924511492281908, 'learning_rate': 0.24122412239323296, 'max_depth': 2, 'n_estimators': 1, 'subsample': 0.9833984902058166}


On teste ensuite le modèle avec les hyperparamètres optimisés sur les données de test.

In [29]:
# On utilise les meilleurs hyperparamètres trouvés par Hyperopt
best_xgb_params = {
    'colsample_bytree': 0.9284956459967804,
    'gamma': 0.04924511492281908,
    'learning_rate': 0.24122412239323296,
    'max_depth': 9,
    'n_estimators': [100, 200, 300, 400, 500][2],  
    'subsample': 0.9833984902058166
}

# Initialiser le modèle XGBoost avec les paramètres optimaux
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=7, **best_xgb_params, random_state=42)

# Entraîner le modèle sur l'ensemble d'entraînement
xgb_model.fit(X_train, y_train_adj)

# Prédire les classes sur l'ensemble de test
y_pred_xgb = xgb_model.predict(X_test)

# Évaluer les performances du modèle
accuracy_xgb = accuracy_score(y_test_adj, y_pred_xgb)
print("Précision du modèle XGBoost :", accuracy_xgb)
print("Rapport de classification XGBoost :\n", classification_report(y_test_adj, y_pred_xgb))

Précision du modèle XGBoost : 0.8862433862433863
Rapport de classification XGBoost :
               precision    recall  f1-score   support

           0       0.81      0.79      0.80       648
           1       0.80      0.73      0.77       648
           2       0.88      0.90      0.89       648
           3       0.96      0.97      0.97       648
           4       0.91      0.95      0.93       648
           5       0.89      0.92      0.90       648
           6       0.94      0.94      0.94       648

    accuracy                           0.89      4536
   macro avg       0.88      0.89      0.89      4536
weighted avg       0.88      0.89      0.89      4536



## LightGBM

In [30]:
import lightgbm as lgb

# On définit l'espace de recherche pour LightGBM
space_lgb = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'max_depth': hp.choice('max_depth', [5, 10, 15, 20, -1]),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'num_leaves': hp.choice('num_leaves', [31, 50, 70, 100]),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

# On crée la fonction objectif pour LightGBM
def objective_lgb(params):
    # On initialise LightGBM avec les paramètres proposés
    model = lgb.LGBMClassifier(objective='multiclass', **params, verbosity = -1,random_state=42)
    # On utilise la validation croisée pour évaluer la précision
    score = cross_val_score(model, X_train, y_train, cv=7, scoring='accuracy').mean()
    # On retourne la perte (score négatif) car Hyperopt minimise cette fonction
    return {'loss': -score, 'status': STATUS_OK}

# On lance l'optimisation avec Hyperopt pour LightGBM
trials_lgb = Trials()
best_params_lgb = fmin(fn=objective_lgb, space=space_lgb, algo=tpe.suggest, max_evals=50, trials=trials_lgb)
print("Meilleurs paramètres LightGBM :", best_params_lgb)

100%|██████████| 50/50 [53:20<00:00, 64.01s/trial, best loss: -0.8912509448223733]  
Meilleurs paramètres LightGBM : {'colsample_bytree': 0.9488048655607615, 'learning_rate': 0.09224159033088929, 'max_depth': 2, 'n_estimators': 2, 'num_leaves': 3, 'subsample': 0.5190158468602316}


On teste ensuite le modèle avec les hyperparamètres optimisés sur les données de test.

In [34]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

# Meilleurs paramètres de LightGBM (après avoir traduit les indices en valeurs)
best_lgb_params = {
    'colsample_bytree': 0.9488048655607615,
    'learning_rate': 0.09224159033088929,
    'max_depth': 15,
    'n_estimators': [100, 200, 300, 400, 500][2],  
    'num_leaves': [31, 50, 70, 100][3],             
    'subsample': 0.5190158468602316,
    'verbosity': -1  # Désactive les avertissements pendant l'entraînement
}

# Initialiser le modèle LightGBM avec les paramètres optimisés
lgb_model = lgb.LGBMClassifier(objective='multiclass', num_class=7, **best_lgb_params, random_state=42)

# Entraîner le modèle sur l'ensemble d'entraînement
lgb_model.fit(X_train, y_train)

# Prédire les classes sur l'ensemble de test
y_pred_lgb = lgb_model.predict(X_test)

# Évaluer les performances du modèle
accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
print("Précision du modèle LightGBM :", accuracy_lgb)
print("Rapport de classification LightGBM :\n", classification_report(y_test, y_pred_lgb))

Précision du modèle LightGBM : 0.8952821869488536
Rapport de classification LightGBM :
               precision    recall  f1-score   support

           1       0.82      0.81      0.81       648
           2       0.82      0.75      0.78       648
           3       0.89      0.90      0.89       648
           4       0.96      0.98      0.97       648
           5       0.93      0.96      0.95       648
           6       0.90      0.92      0.91       648
           7       0.94      0.96      0.95       648

    accuracy                           0.90      4536
   macro avg       0.89      0.90      0.89      4536
weighted avg       0.89      0.90      0.89      4536

